In [2]:
import pandas as pd
import numpy as np
import re
import os

# 1. Load Data
file_path = 'NZ_data.csv'
if not os.path.exists(file_path):
    print(f"ERROR: '{file_path}' not found.")
    raise FileNotFoundError("Missing Data File")

try:
    df = pd.read_csv(file_path, sep=';')
except Exception as e:
    raise

# 2. Clean Data
def clean_numeric(val):
    if isinstance(val, str):
        val = val.split(',')[0]
    return float(val)

def clean_ir(val):
    if pd.isna(val): return np.nan
    val = str(val)
    parts = val.split(',')
    if len(parts) >= 2:
        integer_part = parts[0]
        decimal_digits = re.sub(r'[^\d]', '', parts[1])
        if len(decimal_digits) > 0:
            return float(f"{integer_part}.{decimal_digits[:2]}")
        return float(integer_part)
    return float(val)

df['GDP_clean'] = df['GDP'].apply(clean_numeric)
df['Cons_clean'] = df['Private_Final_Consumption'].apply(clean_numeric)
df['IR_clean'] = df['M3_T_IR'].apply(clean_ir)

# 3. Rename columns for Dynare (r -> R_rate)
df['gy'] = np.log(df['GDP_clean']).diff()
df['gc'] = np.log(df['Cons_clean']).diff()
df['R_rate'] = 1 + (df['IR_clean'] / 100) / 4

# 4. Save
final_df = df[['gy', 'gc', 'R_rate']].dropna()
final_df.to_csv('macro_data_NZ.csv', index=False)
print("SUCCESS: Data saved with columns:", final_df.columns.tolist())


SUCCESS: Data saved with columns: ['gy', 'gc', 'R_rate']
